# Stock Price Perdiction Model

In [2]:
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [3]:
### show pakages vision 
print("Yahoo Fianance lib : ", yf.__version__)
print("Panadas : ", pd.__version__)

Yahoo Fianance lib :  0.1.63
Panadas :  1.3.5


In [4]:
stock_symbol = "GOOGL"
start_date = "2022-01-01"
end_date = "2023-01-01"
# Download historical stock data
data = yf.download(stock_symbol, start="2022-01-01", end="2023-01-01")
data


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,145.054993,145.850998,143.712997,144.991501,144.991501,28646000
2022-01-04,145.395996,146.485001,143.716507,144.399506,144.399506,28400000
2022-01-05,144.419998,144.499496,137.688004,137.774994,137.774994,54618000
2022-01-06,136.998505,139.940002,136.558502,137.747498,137.747498,37348000
2022-01-07,138.145493,138.448502,135.766495,137.016998,137.016998,29760000
...,...,...,...,...,...,...
2022-12-23,87.110001,89.550003,87.070000,89.230003,89.230003,23003000
2022-12-27,88.800003,88.940002,87.010002,87.389999,87.389999,20097300
2022-12-28,86.980003,88.040001,85.940002,86.019997,86.019997,19523200


In [5]:
print(data.isnull().sum())

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [6]:
# Calculate daily returns using the closing prices
data["Daily_Return"] = data["Close"].pct_change()
data

,Open,High,Low,Close,Adj Close,Volume,Daily_Return
Date,,,,,,,
2022-01-03,145.054993,145.850998,143.712997,144.991501,144.991501,28646000,NaN
2022-01-04,145.395996,146.485001,143.716507,144.399506,144.399506,28400000,-0.004083
2022-01-05,144.419998,144.499496,137.688004,137.774994,137.774994,54618000,-0.045876
2022-01-06,136.998505,139.940002,136.558502,137.747498,137.747498,37348000,-0.000200
2022-01-07,138.145493,138.448502,135.766495,137.016998,137.016998,29760000,-0.005303
...,...,...,...,...,...,...,...
2022-12-23,87.110001,89.550003,87.070000,89.230003,89.230003,23003000,0.016750
2022-12-27,88.800003,88.940002,87.010002,87.389999,87.389999,20097300,-0.020621
2022-12-28,86.980003,88.040001,85.940002,86.019997,86.019997,19523200,-0.015677


In [7]:
# Calculate moving averages as additional features
data["MA_10"] = data["Close"].rolling(window=10).mean()
data["MA_50"] = data["Close"].rolling(window=50).mean()
data[10:70]


,Open,High,Low,Close,Adj Close,Volume,Daily_Return,MA_10,MA_50
Date,,,,,,,,,
2022-01-18,136.175003,137.131500,135.438507,135.998001,135.998001,34872000,-0.024968,139.084048,NaN
2022-01-19,136.523499,137.959503,135.015503,135.116501,135.116501,28648000,-0.006482,138.155748,NaN
2022-01-20,136.250000,137.626007,132.964493,133.307495,133.307495,29908000,-0.013388,137.708998,NaN
2022-01-21,132.593506,134.865494,130.086502,130.351501,130.351501,55652000,-0.022174,136.969398,NaN
2022-01-24,125.977997,131.203003,124.500000,130.804001,130.804001,76622000,0.003471,136.348099,NaN
2022-01-25,128.740005,129.399002,126.500504,126.934998,126.934998,46960000,-0.029579,135.174649,NaN
2022-01-26,131.119003,133.000000,127.141998,129.233002,129.233002,49130000,0.018104,134.124350,NaN
2022-01-27,131.304001,132.652496,128.932495,129.005005,129.005005,31950000,-0.001764,132.881801,NaN
2022-01-28,129.658997,133.356506,128.485001,133.350998,133.350998,34362000,0.033689,132.358200,NaN


In [8]:

def calculate_rsi(data, window=14):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

In [13]:
# Calculate the relative strength index (RSI) as another feature

data["RSI"] = calculate_rsi(data["Close"], window=14)

# Create a target variable indicating whether the next day's price will increase or decrease
data["Next_Day_Price_Up"] = (data["Close"].shift(-1) > data["Close"]).astype(int)

# Display the cleaned and preprocessed data
data[0:50]

,Open,High,Low,Close,Adj Close,Volume,Daily_Return,MA_10,MA_50,RSI,Next_Day_Price_Up
Date,,,,,,,,,,,
2022-01-03,145.054993,145.850998,143.712997,144.991501,144.991501,28646000,NaN,NaN,NaN,NaN,0
2022-01-04,145.395996,146.485001,143.716507,144.399506,144.399506,28400000,-0.004083,NaN,NaN,0.000000,0
2022-01-05,144.419998,144.499496,137.688004,137.774994,137.774994,54618000,-0.045876,NaN,NaN,0.000000,0
2022-01-06,136.998505,139.940002,136.558502,137.747498,137.747498,37348000,-0.000200,NaN,NaN,0.000000,0
2022-01-07,138.145493,138.448502,135.766495,137.016998,137.016998,29760000,-0.005303,NaN,NaN,0.000000,1
2022-01-10,135.078003,138.819504,133.164505,138.669495,138.669495,44408000,0.012061,NaN,NaN,17.165228,1
2022-01-11,138.007004,140.216003,136.692505,139.735992,139.735992,28730000,0.007691,NaN,NaN,25.426614,1
2022-01-12,141.149994,142.608002,140.694504,141.430496,141.430496,26108000,0.012126,NaN,NaN,35.627202,0
2022-01-13,141.539993,142.850006,138.408997,138.587006,138.587006,31436000,-0.020105,NaN,NaN,28.976138,1


In [ ]:
### some of feature are calculate from pervious are 
